***GENERATED CODE FOR bidsynergy17kclassify PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'NoticeId', 'transformation_label': 'String Indexer'}], 'feature': 'NoticeId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '003916e9c76d4f9c88b6a8873d361e24', 'max': 'fe4872cb4be447059dbde6f712e86619', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'NoticeId'}, {'feature_label': 'NoticeId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('NoticeId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Department_Ind_Agency', 'transformation_label': 'String Indexer'}], 'feature': 'Department_Ind_Agency', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AGENCY FOR INTERNATIONAL DEVELOPMENT', 'max': 'VETERANS AFFAIRS, DEPARTMENT OF', 'missing': '0', 'distinct': '21'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Department_Ind_Agency'}, {'feature_label': 'Department_Ind_Agency', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Department_Ind_Agency')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'AwardDate', 'transformation_label': 'String Indexer'}], 'feature': 'AwardDate', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '94', 'mean': '', 'stddev': '', 'min': '1/25/2024', 'max': '4/1/2024', 'missing': '406', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'AwardDate'}, {'feature_label': 'AwardDate', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('AwardDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Award$', 'transformation_label': 'String Indexer'}], 'feature': 'Award$', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '107', 'mean': '4889524.26', 'stddev': '31363459.79', 'min': '0', 'max': '99499.65', 'missing': '393', 'distinct': '94'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Award$'}, {'feature_label': 'Award$', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Award$')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Awardee', 'transformation_label': 'String Indexer'}], 'feature': 'Awardee', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '168', 'mean': '', 'stddev': '', 'min': 'AERO-DYNE SUPPLY CO., INC. ', 'max': 'null ', 'missing': '332', 'distinct': '79'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Awardee'}, {'feature_label': 'Awardee', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Awardee')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryContactFullname', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryContactFullname', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '497', 'mean': '', 'stddev': '', 'min': '2nd Lt Summer Guess', 'max': 'scott.reed2@va.gov', 'missing': '2', 'distinct': '366'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryContactFullname'}, {'feature_label': 'PrimaryContactFullname', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryContactFullname')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryContactEmail', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryContactEmail', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '495', 'mean': '', 'stddev': '', 'min': '422.SCMS.AFMC.RFI@us.af.mil.', 'max': 'zacharyalexander.mitchellperez.civ@us.navy.mil', 'missing': '5', 'distinct': '361'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryContactEmail'}, {'feature_label': 'PrimaryContactEmail', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryContactEmail')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryContactPhone', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryContactPhone', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '255', 'mean': '1375601545861.18', 'stddev': '10464762108681.71', 'min': '(202)674-8618', 'max': 'n/A', 'missing': '245', 'distinct': '203'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryContactPhone'}, {'feature_label': 'PrimaryContactPhone', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryContactPhone')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Description', 'transformation_label': 'String Indexer'}], 'feature': 'Description', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '429', 'mean': '', 'stddev': '', 'min': '"02 Apr 2024 - This posting has been edited to include RFP Amendment 0002 which: Extends the due date of proposals to 30 April at 3:30 PM EST Rescinds ""ATCH-01 MAAS - Statement Of Work (SOW)"" and replaces in its entirety with a revised version to add the timeframe for Government generated test reports in SOW Para 3.5.2.2. Rescinds ""EXHBIT A CDRLs A001-A015 DD 1423s"" and replaces in its entirety with a revised version to incorporate changes to CDRL A009. Rescinds ""EXHIBIT A CDRL A016 First Article Test Report - DD 1423"" and replaces its entirety with a revised version. 02 April 2024 - This listing has been edited to add ""Question and Answers 2"" dated 2 Apr 2024 13 Mar 2024- This posting has been edited to add\xa0Questions and Answers dated 13 Mar 2024 12 Mar 2024 - This posting has been edited to include RFP Amendment 0001 which: Extends the due date of proposals to 15 April at 3:30 PM EST. Corrects the First Article Required Delivery. Rescinds ""ATCH-02 Instructions Conditions and Notices to Offerors"" with a revised copy to correct information regarding the *PPI Tool and corrects references to attachment numbers. Adds attachment ""ATCH-26 - NAVAIR Test Matrix"". Removes clause 52.244-2',
                                                                                                                                                                                                                                                                        'max': '“THIS IS A NON-WORK PLAN TASK ORDER RFP TO MATOC CONTRACT, AREA IV ONLY” Please see attached the solicitation and drawing for more details. Site Visit Scheduled for 9 April 2024 at 1100 at in from of Bldg #835, Camp Carroll.', 'missing': '71', 'distinct': '410'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Description'}, {'feature_label': 'Description', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Description')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run bidsynergy17kclassifyHooks.ipynb
try:
	#sourcePreExecutionHook()

	contractopportunitiescsvkrows = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/ContractOpportunitiesCSV3_17K Rows.csv', 'filename': 'ContractOpportunitiesCSV3_17K Rows.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(contractopportunitiescsvkrows)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run bidsynergy17kclassifyHooks.ipynb
try:
	#transformationPreExecutionHook()

	bidsynergykclassifyautofe = TransformationMain.run(contractopportunitiescsvkrows,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "NoticeId", "transformation_label": "String Indexer"}], "feature": "NoticeId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "003916e9c76d4f9c88b6a8873d361e24", "max": "fe4872cb4be447059dbde6f712e86619", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "NoticeId"}, {"transformationsData": [{"feature_label": "Department_Ind_Agency", "transformation_label": "String Indexer"}], "feature": "Department_Ind_Agency", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AGENCY FOR INTERNATIONAL DEVELOPMENT", "max": "VETERANS AFFAIRS, DEPARTMENT OF", "missing": "0", "distinct": "21"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Department_Ind_Agency"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NaicsCode", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "491", "mean": "399186.71", "stddev": "160538.32", "min": "4241", "max": "812990", "missing": "67"}, "updatedLabel": "NaicsCode"}, {"transformationsData": [{"feature_label": "AwardDate", "transformation_label": "String Indexer"}], "feature": "AwardDate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "94", "mean": "", "stddev": "", "min": "1/25/2024", "max": "4/1/2024", "missing": "406", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "AwardDate"}, {"transformationsData": [{"feature_label": "Award$", "transformation_label": "String Indexer"}], "feature": "Award$", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "107", "mean": "4889524.26", "stddev": "31363459.79", "min": "0", "max": "99499.65", "missing": "393", "distinct": "94"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Award$"}, {"transformationsData": [{"feature_label": "Awardee", "transformation_label": "String Indexer"}], "feature": "Awardee", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "168", "mean": "", "stddev": "", "min": "AERO-DYNE SUPPLY CO., INC. ", "max": "null ", "missing": "332", "distinct": "79"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Awardee"}, {"transformationsData": [{"feature_label": "PrimaryContactFullname", "transformation_label": "String Indexer"}], "feature": "PrimaryContactFullname", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "497", "mean": "", "stddev": "", "min": "2nd Lt Summer Guess", "max": "scott.reed2@va.gov", "missing": "2", "distinct": "366"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryContactFullname"}, {"transformationsData": [{"feature_label": "PrimaryContactEmail", "transformation_label": "String Indexer"}], "feature": "PrimaryContactEmail", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "495", "mean": "", "stddev": "", "min": "422.SCMS.AFMC.RFI@us.af.mil.", "max": "zacharyalexander.mitchellperez.civ@us.navy.mil", "missing": "5", "distinct": "361"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryContactEmail"}, {"transformationsData": [{"feature_label": "PrimaryContactPhone", "transformation_label": "String Indexer"}], "feature": "PrimaryContactPhone", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "255", "mean": "1375601545861.18", "stddev": "10464762108681.71", "min": "(202)674-8618", "max": "n/A", "missing": "245", "distinct": "203"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryContactPhone"}, {"transformationsData": [{"feature_label": "Description", "transformation_label": "String Indexer"}], "feature": "Description", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "429", "mean": "", "stddev": "", "min": "\"02 Apr 2024 - This posting has been edited to include RFP Amendment 0002 which: Extends the due date of proposals to 30 April at 3:30 PM EST Rescinds \"\"ATCH-01 MAAS - Statement Of Work (SOW)\"\" and replaces in its entirety with a revised version to add the timeframe for Government generated test reports in SOW Para 3.5.2.2. Rescinds \"\"EXHBIT A CDRLs A001-A015 DD 1423s\"\" and replaces in its entirety with a revised version to incorporate changes to CDRL A009. Rescinds \"\"EXHIBIT A CDRL A016 First Article Test Report - DD 1423\"\" and replaces its entirety with a revised version. 02 April 2024 - This listing has been edited to add \"\"Question and Answers 2\"\" dated 2 Apr 2024 13 Mar 2024- This posting has been edited to add\u00a0Questions and Answers dated 13 Mar 2024 12 Mar 2024 - This posting has been edited to include RFP Amendment 0001 which: Extends the due date of proposals to 15 April at 3:30 PM EST. Corrects the First Article Required Delivery. Rescinds \"\"ATCH-02 Instructions Conditions and Notices to Offerors\"\" with a revised copy to correct information regarding the *PPI Tool and corrects references to attachment numbers. Adds attachment \"\"ATCH-26 - NAVAIR Test Matrix\"\". Removes clause 52.244-2", "max": "\u201cTHIS IS A NON-WORK PLAN TASK ORDER RFP TO MATOC CONTRACT, AREA IV ONLY\u201d Please see attached the solicitation and drawing for more details. Site Visit Scheduled for 9 April 2024 at 1100 at in from of Bldg #835, Camp Carroll.", "missing": "71", "distinct": "410"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Description"}]}))

	#transformationPostExecutionHook(bidsynergykclassifyautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run bidsynergy17kclassifyHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(bidsynergykclassifyautofe, ["NaicsCode", "NoticeId_stringindexer", "AwardDate_stringindexer", "Award$_stringindexer", "Awardee_stringindexer", "PrimaryContactFullname_stringindexer", "PrimaryContactEmail_stringindexer", "PrimaryContactPhone_stringindexer", "Description_stringindexer"], "Department_Ind_Agency_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

